In [1]:
import pandas as pd
import gensim.corpora as corpora
import spacy
import gensim
import nltk
from nltk.tokenize import word_tokenize

C:\Users\dario\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv("abortionNews.csv")

In [4]:
text = "This is a string" 
tokens = word_tokenize(text.replace('-', ' '))

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
texts = list(sent_to_words(df['content']))

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
data_words_nostops = remove_stopwords(texts)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [16]:
clean_text = []

for tokens in list(data_lemmatized):
    clean_text.append(" ".join(tokens)) 

In [17]:
df["clean_text"] = clean_text

In [28]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [36]:
scores = sia.polarity_scores("Wow, NLTK is really powerful!")

In [ ]:
## Get sentiments
def get_sentiments(input_text):
    labels = []
    for text in input_text:
        sentiment = sia.polarity_scores(text)
        score = sentiment.get('compound')
        if score > 0.5:
            labels.append("positive")
        elif score < -0.5:
            labels.append("negative")
        else:
            labels.append("neutral")
    return labels

news_sentiment = get_sentiments(df["clean_text"])

df["sentiment"] = news_sentiment

In [53]:
df["sentiment"].value_counts()